# Script for a period

In [35]:
import requests
import json
import pandas as pd
from datetime import date
from clickhouse_connect import get_client
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import time


load_dotenv()

True

In [36]:
KeyGuten = os.getenv('KeyGuten')
KeyGiper = os.getenv('KeyGiper')
KeyKitchen = os.getenv('KeyKitchen')
KeySmart = os.getenv("KeySmart")

## Списки кампаний

Link: https://dev.wildberries.ru/openapi/promotion#tag/Kampanii/paths/~1adv~1v1~1promotion~1count/get

In [37]:


# API endpoint
url = 'https://advert-api.wildberries.ru/adv/v1/promotion/count'

# Headers including the API key for authentication
headers_guten = {
    'Authorization': KeyGuten,
    'Accept': 'application/json'
}

headers_giper = {
    'Authorization': KeyGiper,
    'Accept': 'application/json'
}

headers_kitchen = {
    'Authorization': KeyKitchen,
    'Accept': 'application/json'
}

headers_smart = {
    'Authorization': KeySmart,
    'Accept': 'application/json'
}

# Make the GET request
response_guten = requests.get(url, headers=headers_guten)
response_giper = requests.get(url, headers=headers_giper)
response_kitchen = requests.get(url, headers=headers_kitchen)
response_smart = requests.get(url, headers=headers_smart)

# Check if the request was successful
if response_guten.status_code == 200 and response_giper.status_code == 200 and response_kitchen.status_code == 200 and response_smart.status_code == 200:
    # Parse the JSON response
    data_guten = response_guten.json()
    data_giper = response_giper.json()
    data_kitchen = response_kitchen.json()
    data_smart = response_smart.json()
    print("Data retrieved successfully")
else:
    print(f"Failed to retrieve data. Status code: {response_guten.status_code,response_giper.status_code,response_giper.status_code,response_smart.status_code }")
    print(f"Response: {response_guten.text, response_giper.text,response_giper.text,response_smart.text }")

Data retrieved successfully


In [38]:
data_giper

{'adverts': [{'type': 9,
   'status': 7,
   'count': 1,
   'advert_list': [{'advertId': 22954217,
     'changeTime': '2025-01-31T16:08:49.754418+03:00'}]},
  {'type': 8,
   'status': 11,
   'count': 4,
   'advert_list': [{'advertId': 21526558,
     'changeTime': '2025-02-07T16:43:04.178341+03:00'},
    {'advertId': 22844037, 'changeTime': '2025-01-31T13:40:18.926964+03:00'},
    {'advertId': 22844946, 'changeTime': '2025-03-04T12:36:41.470052+03:00'},
    {'advertId': 23402220, 'changeTime': '2025-03-04T13:29:43.833312+03:00'}]},
  {'type': 9,
   'status': 9,
   'count': 5,
   'advert_list': [{'advertId': 22950405,
     'changeTime': '2025-03-03T12:03:56.844328+03:00'},
    {'advertId': 22950831, 'changeTime': '2025-03-03T12:28:57.920554+03:00'},
    {'advertId': 22951156, 'changeTime': '2025-03-03T15:35:29.753881+03:00'},
    {'advertId': 22951268, 'changeTime': '2025-02-07T16:45:41.628974+03:00'},
    {'advertId': 22954069, 'changeTime': '2025-02-07T16:05:53.370485+03:00'}]},
  {'typ

In [39]:
# Flatten the JSON data
df_guten = pd.json_normalize(
    data_guten['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_giper = pd.json_normalize(
    data_giper['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_kitchen = pd.json_normalize(
    data_kitchen['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_smart = pd.json_normalize(
    data_smart['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)

# Convert 'changeTime' to datetime format
df_guten['changeTime'] = pd.to_datetime(df_guten['changeTime'])
df_giper['changeTime'] = pd.to_datetime(df_giper['changeTime'])
df_kitchen['changeTime'] = pd.to_datetime(df_kitchen['changeTime'])
df_smart['changeTime'] = pd.to_datetime(df_smart['changeTime'])

# Filter rows where the year of 'changeTime' is 2024 or later
#df_guten = df_guten[df_guten['changeTime'].dt.year >= 2024]
#df_giper = df_giper[df_giper['changeTime'].dt.year >= 2024]
#df_kitchen = df_kitchen[df_kitchen['changeTime'].dt.year >= 2024]
#df_smart = df_smart[df_smart['changeTime'].dt.year >= 2024]

# Reset the index and drop the old index
df_guten = df_guten.reset_index(drop=True)
df_giper = df_giper.reset_index(drop=True)
df_kitchen = df_kitchen.reset_index(drop=True)
df_smart = df_smart.reset_index(drop=True)

# Display the updated DataFrame
#df_guten
#df_giper
#df_kitchen

In [40]:
#unique_change_times = df_guten['advertId'].nunique()
#print(unique_change_times)

### Информация о кампаниях


Link: https://dev.wildberries.ru/openapi/promotion#tag/Kampanii/paths/~1adv~1v1~1promotion~1adverts/post

In [41]:
# Create chunks of 50 campaign IDs each
chunk_size = 50
campaign_chunks_guten = [df_guten['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_guten), chunk_size)]
campaign_chunks_giper = [df_giper['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_giper), chunk_size)]
campaign_chunks_kitchen = [df_kitchen['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_kitchen), chunk_size)]
campaign_chunks_smart = [df_smart['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_smart), chunk_size)]

# Print the list of all chunks
print("Campaign Chunks for WB-GutenTech:", campaign_chunks_guten)
print("Campaign Chunks for WB-ГиперМаркет:", campaign_chunks_giper)
print("Campaign Chunks for WB-KitchenAid:", campaign_chunks_kitchen)
print("Campaign Chunks for WB-Smart-Market:", campaign_chunks_smart)

# Define the API endpoint
url = "https://advert-api.wildberries.ru/adv/v1/promotion/adverts"

# Define the query parameters
query_params = {
    "order": "create",  # Order by the "change" field
    "direction":"desc"
}


# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, project_name):
    all_campaign_data = []
    for idx, chunk in enumerate(chunks):
        response = requests.post(url, params=query_params, json=chunk, headers=headers)
        if response.status_code == 200:
            data = response.json()
            all_campaign_data.extend(data)
            print(f"Data retrieved successfully for {project_name} chunk {idx + 1}")
        else:
            print(f"Error for {project_name} chunk {idx + 1}: {response.status_code}, {response.text}")
        time.sleep(1)  # Add a delay to avoid hitting API rate limits
    return all_campaign_data

# Fetch campaign data for each project
all_campaign_data_guten = fetch_campaign_data(campaign_chunks_guten, headers_guten, 'WB-GutenTech')
all_campaign_data_giper = fetch_campaign_data(campaign_chunks_giper, headers_giper, 'WB-ГиперМаркет')
all_campaign_data_kitchen = fetch_campaign_data(campaign_chunks_kitchen, headers_kitchen, 'WB-KitchenAid')
all_campaign_data_smart = fetch_campaign_data(campaign_chunks_smart, headers_smart, 'WB-Smart-Market')

# Create DataFrames from the campaign data
campaigns_guten = pd.DataFrame(all_campaign_data_guten)
campaigns_giper = pd.DataFrame(all_campaign_data_giper)
campaigns_kitchen = pd.DataFrame(all_campaign_data_kitchen)
campaigns_smart = pd.DataFrame(all_campaign_data_smart)

# Sort each DataFrame by 'createTime' in descending order
campaigns_guten = campaigns_guten.sort_values(by='createTime', ascending=False)
campaigns_giper = campaigns_giper.sort_values(by='createTime', ascending=False)
campaigns_kitchen = campaigns_kitchen.sort_values(by='createTime', ascending=False)
campaigns_smart = campaigns_smart.sort_values(by='createTime', ascending=False)

campaigns_guten['Project'] = 'WB-GutenTech'
campaigns_giper['Project'] = 'WB-ГиперМаркет'
campaigns_kitchen['Project'] = 'WB-KitchenAid'
campaigns_smart['Project'] = 'WB-Smart-Market'

# Concatenate the DataFrames
combined_campaigns = pd.concat([campaigns_guten, campaigns_giper, campaigns_kitchen,campaigns_smart], ignore_index=True)

combined_campaigns['Marketplace'] = 'Wildberries'

print("Combined Campaign Data")
#campaigns_kitchen

Campaign Chunks for WB-GutenTech: [[213920, 213946, 213954, 225156, 232543, 232547, 238682, 244654, 244667, 354788, 354832, 354853, 535702, 535721, 535733, 535812, 535864, 535918, 535940, 535963, 535994, 536002, 536018, 536060, 536362, 536369, 536385, 536415, 536427, 536485, 536514, 536537, 536549, 536568, 536610, 536617, 536631, 536646, 536659, 788466, 788472, 788580, 788586, 788636, 788641, 789178, 789191, 953420, 953764, 953774], [958835, 960520, 960525, 960529, 960544, 960551, 960611, 960618, 960621, 960624, 960626, 960675, 960682, 960689, 960693, 960701, 1005700, 1320657, 1320668, 1320684, 2594125, 2594226, 2594682, 2594752, 2595105, 2595113, 2851221, 2851300, 2851888, 2852058, 3467203, 3474308, 3474441, 3476519, 3476542, 3476739, 3476757, 3476880, 3476891, 3477028, 4558192, 4981861, 5001772, 5146248, 5146392, 5149382, 6626942, 6923910, 6923977, 7202606], [7202975, 7203114, 7339607, 7339693, 7339737, 7339839, 7339928, 7339962, 7340006, 7340088, 7414147, 8779205, 8779248, 8779408, 

In [113]:
# Keep only the desired columns
columns_to_keep = ["endTime", "createTime", "startTime","name", "advertId", "status", "type","Project","Marketplace"]
# Ensure filtered_df is a copy of the slice, not a view
filtered_df = combined_campaigns[columns_to_keep].copy()
filtered_df['endTime'] = pd.to_datetime(filtered_df['endTime'], format='mixed').dt.date
filtered_df['createTime'] = pd.to_datetime(filtered_df['createTime'], format='mixed').dt.date
filtered_df['startTime'] = pd.to_datetime(filtered_df['startTime'], format='mixed').dt.date

# Mapping dictionaries for 'status' and 'type'
status_mapping = {
    -1: "Кампания в процессе удаления",
    4: "Готова к запуску",
    7: "Кампания завершена",
    8: "Отказался",
    9: "Идут показы",
    11: "Кампания на паузе"
}

type_mapping = {
    4: "Кампания в каталоге (устаревший тип)",
    5: "Кампания в карточке товара (устаревший тип)",
    6: "Кампания в поиске (устаревший тип)",
    7: "Кампания в рекомендациях на главной странице (устаревший тип)",
    8: "Автоматическая кампания",
    9: "Аукцион"
}

# Replace numeric values with their string descriptions
filtered_df['status'] = filtered_df['status'].replace(status_mapping)
filtered_df['type'] = filtered_df['type'].replace(type_mapping)

# Display the updated DataFrame
filtered_df

,endTime,createTime,startTime,name,advertId,status,type,Project,Marketplace
0,2100-01-01,2025-02-19,2025-02-19,Mr Proper_19.02.2025,23408417,Кампания на паузе,Автоматическая кампания,WB-GutenTech,Wildberries
1,2100-01-01,2025-02-12,2025-03-13,Venus_12.02.2025,23231292,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
2,2100-01-01,2025-02-12,2025-03-10,Always_12.02.2025,23230882,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
3,2100-01-01,2025-01-27,2025-01-27,AND_Тонометры на запястье_27.01.2025,22846531,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
4,2100-01-01,2025-01-27,2025-03-12,AND_Тонометры авто_27.01.2025,22846516,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...
470,2024-06-23,2024-03-21,2024-04-18,Насадки Oral-B Автореклама,15562382,Кампания завершена,Автоматическая кампания,WB-Smart-Market,Wildberries
471,2100-01-01,2024-03-21,2025-02-20,Braun_Электробритвы_22.10.2024,15562363,Идут показы,Автоматическая кампания,WB-Smart-Market,Wildberries
472,2024-10-11,2024-03-21,2024-07-02,Электрические зубные щетки Oral-B Поиск + Каталог,15562321,Кампания завершена,Аукцион,WB-Smart-Market,Wildberries
473,2024-10-15,2024-03-21,2024-08-27,Эпиляторы Braun_7 SKU_27.08.2024,15562282,Кампания завершена,Автоматическая кампания,WB-Smart-Market,Wildberries


## Статистика кампаний


Link: https://dev.wildberries.ru/openapi/analytics#tag/Statistika-po-prodvizheniyu/paths/~1adv~1v2~1fullstats/post

Данные вернутся для кампаний в статусах:

* 7 — завершена
* 9 — идут показы
* 11 — пауза из-за расхода бюджета

Response Schema: 

* dates	/ Array of strings <date> [ items <date > ]/ Даты, за которые необходимо выдать информацию.
* views	/ integer/ Количество просмотров./ За все дни, по всем артикулам WB и платформам.
* clicks	/ integer/ Количество кликов./ За все дни, по всем артикулам WB и платформам.
* ctr	/ number/ Показатель кликабельности./ Отношение числа кликов к количеству показов. Выражается в процентах./ За все дни, по всем артикулам WB и платформам.
* cpc	/ number/ Средняя стоимость клика, ₽./ За все дни, по всем артикулам WB и платформам.
* sum	/ number/ Затраты, ₽./ За все дни, по всем артикулам WB и платформам.
* atbs	/ integer/ Количество добавлений товаров в корзину./ За все дни, по всем артикулам WB и платформам.
* orders	/ integer/ Количество заказов./ За все дни, по всем артикулам WB и платформам.
* cr	/ integer/ CR(conversion rate) — это отношение количества заказов к общему количеству посещений кампании./ За все дни, по всем артикулам WB и платформам.
* shks	/ integer/ Количество заказанных товаров, шт./ За все дни, по всем артикулам WB и платформам.
* sum_price	/ number/ Заказов на сумму, ₽/ За все дни, по всем артикулам WB и платформам.
* days	/ Array of objects (Days)/ Статистка по дням
    Array 
    * date	/ string <date-time>/ Дата, за которую представлены данные/ 
    * views	/ integer/ Количество просмотров/ 
    * clicks	/ integer/ Количество кликов/ 
    * ctr	/ number/ Показатель кликабельности, отношение числа кликов к количеству показов, %/ 
    * cpc	/ number/ Средняя стоимость клика, ₽./ 
    * sum	/ number/ Затраты, ₽./ 
    * atbs	/ integer/ Количество добавлений товаров в корзину/ 
    * orders	/ integer/ Количество заказов/ 
    * cr	/ integer/ CR(conversion rate) — отношение количества заказов к общему количеству посещений кампании/ 
    * shks	/ integer/ Количество заказанных товаров, шт./ 
    * sum_price	/ number/ Заказов на сумму, ₽/ 
    * apps	/ Array of objects/ Блок информации о платформе
    Array 
        - views	/ integer/ Количество просмотров/ 
        - clicks	/ integer/ Количество кликов/ 
        - ctr	/ number/ Показатель кликабельности, отношение числа кликов к количеству показов, %/ 
        - cpc	/ number/ Средняя стоимость клика, ₽./ 
        - sum	/ number/ Затраты, ₽./ 
        - atbs	/ integer/ Количество добавлений товаров в корзину/ 
        - orders	/ integer/ Количество заказов/ 
        - cr	/ integer/ CR(conversion rate) — это отношение количества заказов к общему количеству посещений кампании/ 
        - shks	/ integer/ Количество заказанных товаров, шт./ 
        - sum_price	/ number/ Заказов на сумму, ₽/ 
        - nm	/ Array of objects/ Блок статистики по артикулам WB/ 
        - appType	/ integer/ Тип платформы (1 - сайт, 32 - Android, 64 - IOS)
* boosterStats	/ Array of objects (BoosterStats)/ Статистика по средней позиции товара на страницах поисковой выдачи и каталога (для автоматических кампаний).
* advertId	/ integer/ ID кампании

In [43]:
# API endpoint
url = "https://advert-api.wildberries.ru/adv/v2/fullstats"

# Define the date interval
date_interval = {
    "begin": "2025-03-01",  # Replace with your start date
    "end": "2025-03-10"     # Replace with your end date
}

# Create chunks of 100 campaign IDs each
chunk_size = 100

### Guten

In [44]:
# Assume campaigns_guten DataFrame is already defined and contains 'advertId'
# Create chunks of 100 campaign IDs each
campaign_chunks = [campaigns_guten['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_guten), chunk_size)]

# List to store all campaign data
all_campaign_data_guten = []

# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, date_interval):
    all_data = []
    for idx, chunk in enumerate(chunks):
        # Construct the payload for the current chunk
        payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(url, headers=headers, json=payload)

        # Add a delay to avoid hitting API rate limits
        time.sleep(65)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1}")
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
    return all_data

# Fetch campaign data
all_campaign_data_guten = fetch_campaign_data(campaign_chunks, headers_guten, date_interval)
# Combine all campaign data into a DataFrame
campaign_guten_df = pd.json_normalize(all_campaign_data_guten)

# Save the combined data to a text file
with open("response_data_guten.txt", "w", encoding="utf-8") as file:
    json.dump(all_campaign_data_guten, file, indent=4, ensure_ascii=False)

# Display the combined DataFrame
print("Combined Campaign Data")

Data retrieved successfully for chunk 1
Data retrieved successfully for chunk 2
Error for chunk 3: 400, {"error":"Invalid Params: there are no companies with correct intervals"}
Error for chunk 4: 400, {"error":"Invalid Params: there are no companies with correct intervals"}
Error for chunk 5: 400, {"error":"Invalid Params: there are no companies with correct intervals"}
Combined Campaign Data


### Giper

In [45]:
campaign_chunks = [campaigns_giper['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_giper), chunk_size)]

# List to store all campaign data
all_campaign_data_giper = []

# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, date_interval):
    all_data = []
    for idx, chunk in enumerate(chunks):
        # Construct the payload for the current chunk
        payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(url, headers=headers, json=payload)

        # Add a delay to avoid hitting API rate limits
        time.sleep(65)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1}")
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
    return all_data
# Fetch campaign data
all_campaign_data_giper = fetch_campaign_data(campaign_chunks, headers_giper, date_interval)

Data retrieved successfully for chunk 1


### Kitchen

In [46]:
chunk_size = 100
campaign_chunks = [campaigns_kitchen['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_kitchen), chunk_size)]

# List to store all campaign data
all_campaign_data_kitchen = []

# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, date_interval):
    all_data = []
    for idx, chunk in enumerate(chunks):
        # Construct the payload for the current chunk
        payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(url, headers=headers, json=payload)

        # Add a delay to avoid hitting API rate limits
        time.sleep(65)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1}")
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
    return all_data

# Fetch campaign data
all_campaign_data_kitchen = fetch_campaign_data(campaign_chunks, headers_kitchen, date_interval)


Data retrieved successfully for chunk 1


### Smart

In [47]:
campaign_chunks = [campaigns_smart['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_smart), chunk_size)]

# List to store all campaign data
all_campaign_data_smart = []

# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, date_interval):
    all_data = []
    for idx, chunk in enumerate(chunks):
        # Construct the payload for the current chunk
        payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(url, headers=headers, json=payload)

        # Add a delay to avoid hitting API rate limits
        time.sleep(65)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1}")
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
    return all_data

# Fetch campaign data
all_campaign_data_smart = fetch_campaign_data(campaign_chunks, headers_smart, date_interval)

Data retrieved successfully for chunk 1


In [48]:
campaign_guten_df

,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,days,boosterStats,advertId,interval.begin,interval.end
0,24108,628,2.60,8.54,5363.98,90,24,3.82,25,34179,"[{'date': '2025-03-01T03:00:00+03:00', 'views'...","[{'date': '2025-03-01T00:00:00Z', 'nm': 266556...",21821164,2025-03-01,2025-03-10
1,5,0,0.00,0.00,0.38,0,0,0.00,0,0,"[{'date': '2025-03-01T03:00:00+03:00', 'views'...",NaN,21041194,2025-03-01,2025-03-10
2,8,1,12.50,0.97,0.97,0,0,0.00,0,0,"[{'date': '2025-03-01T03:00:00+03:00', 'views'...","[{'date': '2025-03-03T00:00:00Z', 'nm': 286529...",21577733,2025-03-01,2025-03-10
3,2,0,0.00,0.00,0.40,0,0,0.00,0,0,"[{'date': '2025-03-07T03:00:00+03:00', 'views'...",NaN,21578389,2025-03-01,2025-03-10
4,21637,739,3.42,7.96,5884.79,79,27,3.65,27,176385,"[{'date': '2025-03-01T03:00:00+03:00', 'views'...","[{'date': '2025-03-01T00:00:00Z', 'nm': 266534...",21821064,2025-03-01,2025-03-10
5,60321,2005,3.32,6.75,13525.53,260,48,2.39,48,122731,"[{'date': '2025-03-01T03:00:00+03:00', 'views'...","[{'date': '2025-03-01T00:00:00Z', 'nm': 171562...",21554118,2025-03-01,2025-03-10
6,37976,863,2.27,10.00,8632.65,64,10,1.16,10,83743,"[{'date': '2025-03-01T03:00:00+03:00', 'views'...",NaN,21204193,2025-03-01,2025-03-10
7,1487,98,6.59,1.37,134.38,8,1,1.02,1,3992,"[{'date': '2025-03-01T03:00:00+03:00', 'views'...","[{'date': '2025-03-01T00:00:00Z', 'nm': 181674...",20139198,2025-03-01,2025-03-10
8,20081,369,1.84,6.67,2459.52,11,1,0.27,1,3225,"[{'date': '2025-03-01T03:00:00+03:00', 'views'...","[{'date': '2025-03-01T00:00:00Z', 'nm': 260262...",18870853,2025-03-01,2025-03-10
9,80679,1816,2.25,5.58,10125.41,131,34,1.87,34,38925,"[{'date': '2025-03-01T03:00:00+03:00', 'views'...","[{'date': '2025-03-01T00:00:00Z', 'nm': 200571...",23230882,2025-03-01,2025-03-10


In [86]:
print(campaign_guten_df["advertId"].nunique())

#unique_change_times = df_guten['advertId'].nunique()
#print(unique_change_times)

57


In [95]:
# Flatten the JSON data
flattened_data_guten = []
flattened_data_giper = []
flattened_data_kitchen = []
flattened_data_smart = []


for entry in all_campaign_data_guten:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_guten.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                
for entry in all_campaign_data_giper:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_giper.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })

for entry in all_campaign_data_kitchen:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_kitchen.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                
for entry in all_campaign_data_smart:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_smart.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                

# Create a DataFrame
df_guten = pd.DataFrame(flattened_data_guten)
df_giper = pd.DataFrame(flattened_data_giper)
df_kitchen = pd.DataFrame(flattened_data_kitchen)
df_smart = pd.DataFrame(flattened_data_smart)
# Convert the 'date' column to datetime
df_guten["date"] = pd.to_datetime(df_guten["date"])
df_giper["date"] = pd.to_datetime(df_giper["date"])
df_kitchen["date"] = pd.to_datetime(df_kitchen["date"])
df_smart["date"] = pd.to_datetime(df_smart["date"])
# Remove timezone information (make it timezone-naive)
df_guten["date"] = df_guten["date"].dt.tz_localize(None)
df_giper["date"] = df_giper["date"].dt.tz_localize(None)
df_kitchen["date"] = df_kitchen["date"].dt.tz_localize(None)
df_smart["date"] = df_smart["date"].dt.tz_localize(None)

# Display the DataFrame
df_guten

,date,nmId,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId
0,2025-03-01 03:00:00,266556138,Насадки для зубных щеток Ultra Clean UС02,13,0,0.00,0.00,3.25,0,0,0.00,0,0,21821164
1,2025-03-01 03:00:00,266556141,Насадки для зубных щеток Professional Clean P1...,20,2,10.00,2.04,4.07,1,0,0.00,0,0,21821164
2,2025-03-01 03:00:00,266556149,Насадки для зубных щеток Professional Clean P1С9,19,3,15.79,1.47,4.42,0,0,0.00,0,0,21821164
3,2025-03-01 03:00:00,266556135,Насадки для зубных щеток Ultra Clean UС01,12,0,0.00,0.00,2.67,0,0,0.00,0,0,21821164
4,2025-03-01 03:00:00,266556139,Насадки для зубных щеток Ultra White UW02,10,1,10.00,2.27,2.27,0,0,0.00,0,0,21821164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7156,2025-03-10 03:00:00,46292575,Яйцеварка электрическая автоматическая E 9,95,1,1.05,19.67,19.67,0,0,0.00,0,0,18870619
7157,2025-03-10 03:00:00,46292575,Яйцеварка электрическая автоматическая E 9,437,7,1.60,11.59,81.15,1,0,0.00,0,0,18870619
7158,2025-03-10 03:00:00,209076655,Яйцеварка электрическая автоматическая E 3,985,34,3.45,3.73,126.91,4,1,2.94,1,3922,18870619
7159,2025-03-10 03:00:00,209076655,Яйцеварка электрическая автоматическая E 3,500,33,6.60,1.96,64.65,2,0,0.00,0,0,18870619


In [94]:
# Specify the file name for the Excel file
file_name = "df_guten_data.xlsx"

# Export the DataFrame to an Excel file
df_guten.to_excel(file_name, index=False)

print(f"DataFrame exported to {file_name} successfully.")

DataFrame exported to df_guten_data.xlsx successfully.


In [111]:
# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_guten = (
    df_guten.groupby([df_guten["date"].dt.date, "nmId", "advertId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_giper = (
    df_giper.groupby([df_giper["date"].dt.date, "nmId", "advertId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_kitchen = (
    df_kitchen.groupby([df_kitchen["date"].dt.date, "nmId", "advertId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_smart = (
    df_smart.groupby([df_smart["date"].dt.date, "nmId", "advertId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)


# Rename the 'date' column to 'day' for clarity
df_grouped_guten.rename(columns={"date": "day"}, inplace=True)
df_grouped_giper.rename(columns={"date": "day"}, inplace=True)
df_grouped_kitchen.rename(columns={"date": "day"}, inplace=True)
df_grouped_smart.rename(columns={"date": "day"}, inplace=True)

df_grouped_guten['Project'] = 'WB-GutenTech'
df_grouped_giper['Project'] = 'WB-ГиперМаркет'
df_grouped_kitchen['Project'] = 'WB-KitchenAid'
df_grouped_smart['Project'] = 'WB-Smart-Market'

# Concatenate the DataFrames
df_grouped_combined_campaigns = pd.concat([df_grouped_guten, df_grouped_giper, df_grouped_kitchen, df_grouped_smart], ignore_index=True)
df_grouped_combined_campaigns['Marketplace'] = 'Wildberries'

# Display the grouped DataFrame
df_grouped_combined_campaigns

,nmId,day,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId,Project,Marketplace
0,25623718,2025-03-01 03:00:00,Бритва мужская электрическая S3 300BT с триммером,49,1,1.043333,0.886667,4.10,0,0,0.000000,0,0,21202453,WB-GutenTech,Wildberries
1,25623718,2025-03-01 03:00:00,Бритва мужская электрическая S3 300BT с триммером,362,17,4.586667,5.723333,74.22,2,0,0.000000,0,0,21202745,WB-GutenTech,Wildberries
2,25624340,2025-03-01 03:00:00,Электробритва Series 3 300s,89,4,5.070000,1.653333,7.16,0,0,0.000000,0,0,21202453,WB-GutenTech,Wildberries
3,25624340,2025-03-01 03:00:00,Электробритва Series 3 300s,355,16,3.720000,6.093333,72.78,1,0,0.000000,0,0,21202745,WB-GutenTech,Wildberries
4,25628178,2025-03-01 03:00:00,Бритва мужская электрическая S3 300s,111,3,1.586667,0.566667,8.76,0,0,0.000000,0,0,21202453,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4914,288465481,2025-03-10 03:00:00,Набор Gillette Venus с чехлом для бритвы,1427,93,5.273333,4.076667,208.60,2,1,0.926667,1,1115,23236382,WB-Smart-Market,Wildberries
4915,288465487,2025-03-10 03:00:00,Набор Series Пена для бритья 200мл + Мочалка,1,0,0.000000,0.000000,0.06,0,0,0.000000,0,0,23236144,WB-Smart-Market,Wildberries
4916,313543155,2025-03-10 03:00:00,,1272,18,1.270000,15.510000,164.92,3,2,5.556667,2,6906,23236221,WB-Smart-Market,Wildberries
4917,313543156,2025-03-10 03:00:00,,1542,22,1.120000,17.813333,203.51,4,1,8.333333,1,3664,23236221,WB-Smart-Market,Wildberries


In [109]:
# Specify the file name for the Excel file
file_name = "df_guten_data.xlsx"

# Export the DataFrame to an Excel file
df_grouped_combined_campaigns.to_excel(file_name, index=False)

print(f"DataFrame exported to {file_name} successfully.")

DataFrame exported to df_guten_data.xlsx successfully.


In [114]:
# Merge the grouped DataFrame with the filtered_df to add additional columns
df_final = df_grouped_combined_campaigns.merge(
    filtered_df[["advertId", "endTime", "createTime", "startTime","name", "status", "type"]],
    on="advertId",
    how="left"
)

# Drop the columns 'ctr', 'cpc', and 'cr'
df_final = df_final.drop(columns=["ctr", "cpc", "cr"])

# Rename the columns 'name_x' and 'name_y'
df_final.rename(
    columns={
        "name_x": "name_product",  # Rename 'name_x' to 'name_product'
        "name_y": "name_campaign"  # Rename 'name_y' to 'name_campaign'
    },
    inplace=True
)

# Display the final DataFrame
df_final
df_final_copy = df_final.copy()


In [115]:
df_final

,nmId,day,name_product,views,clicks,sum,atbs,orders,shks,sum_price,advertId,Project,Marketplace,endTime,createTime,startTime,name_campaign,status,type
0,25623718,2025-03-01 03:00:00,Бритва мужская электрическая S3 300BT с триммером,49,1,4.10,0,0,0,0,21202453,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
1,25623718,2025-03-01 03:00:00,Бритва мужская электрическая S3 300BT с триммером,362,17,74.22,2,0,0,0,21202745,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион
2,25624340,2025-03-01 03:00:00,Электробритва Series 3 300s,89,4,7.16,0,0,0,0,21202453,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
3,25624340,2025-03-01 03:00:00,Электробритва Series 3 300s,355,16,72.78,1,0,0,0,21202745,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион
4,25628178,2025-03-01 03:00:00,Бритва мужская электрическая S3 300s,111,3,8.76,0,0,0,0,21202453,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4914,288465481,2025-03-10 03:00:00,Набор Gillette Venus с чехлом для бритвы,1427,93,208.60,2,1,1,1115,23236382,WB-Smart-Market,Wildberries,2100-01-01,2025-02-12,2025-03-05,VENUS_Наборы_12.02.2025,Кампания на паузе,Автоматическая кампания
4915,288465487,2025-03-10 03:00:00,Набор Series Пена для бритья 200мл + Мочалка,1,0,0.06,0,0,0,0,23236144,WB-Smart-Market,Wildberries,2100-01-01,2025-02-12,2025-02-24,Gillette_Бритвы + наборы_12.02.2025,Кампания на паузе,Автоматическая кампания
4916,313543155,2025-03-10 03:00:00,,1272,18,164.92,3,2,2,6906,23236221,WB-Smart-Market,Wildberries,2100-01-01,2025-02-12,2025-02-21,Gillette_Кассеты_12.02.2025,Кампания на паузе,Автоматическая кампания
4917,313543156,2025-03-10 03:00:00,,1542,22,203.51,4,1,1,3664,23236221,WB-Smart-Market,Wildberries,2100-01-01,2025-02-12,2025-02-21,Gillette_Кассеты_12.02.2025,Кампания на паузе,Автоматическая кампания


In [116]:
# Specify the file name for the Excel file
file_name = "df_guten_data.xlsx"

# Export the DataFrame to an Excel file
df_final.to_excel(file_name, index=False)

print(f"DataFrame exported to {file_name} successfully.")

DataFrame exported to df_guten_data.xlsx successfully.


In [63]:
# API endpoint
url = "https://seller-analytics-api.wildberries.ru/api/v2/nm-report/detail/history"

# Assuming filtered_df is already defined and contains the necessary data
# Extract all unique nmId values from the filtered DataFrame
filtered_df = df_final[df_final['Project'] == 'WB-GutenTech']
unique_nmId_values = filtered_df['nmId'].unique().tolist()
print("Total unique nmId values:", len(unique_nmId_values))  # Debugging line

# Function to send a single POST request
def send_request(nm_ids, period, headers):
    request_body = {
        "nmIDs": nm_ids,
        "period": period
    }
    response = requests.post(url, headers=headers, json=request_body)
    return response

# Function to fetch data in batches
def fetch_data_in_batches(nm_ids, period, headers):
    all_data = []
    batch_size = 20  # Maximum allowed nmIDs per request
    requests_per_minute = 3  # Maximum allowed requests per minute
    interval = 60 / requests_per_minute  # Time interval between requests in seconds

    for i in range(0, len(nm_ids), batch_size):
        batch = nm_ids[i:i + batch_size]
        response = send_request(batch, period, headers)

        if response.status_code == 200:
            try:
                data = response.json()
                if not data.get('error') and 'data' in data:
                    all_data.extend(data['data'])
                    print(f"Data retrieved successfully for batch {i // batch_size + 1}")
                else:
                    print(f"Error in response for batch {i // batch_size + 1}: {data.get('errorText', 'No error text')}")
            except ValueError as e:
                print(f"Failed to decode JSON for batch {i // batch_size + 1}: {e}")
        else:
            print(f"Error for batch {i // batch_size + 1}: {response.status_code}, {response.text}")

        time.sleep(interval)  # Respect the rate limit

    return all_data

# Define the period for the report
period = {
    "begin": "2025-03-06",
    "end": "2025-03-11"
}

# Fetch data for all unique nmId values
all_data = fetch_data_in_batches(unique_nmId_values, period, headers_guten)

# Process the data as needed
# Flatten the nested 'history' data for easier analysis
flattened_data = []
for item in all_data:
    nmID = item['nmID']
    imtName = item['imtName']
    vendorCode = item['vendorCode']
    for history in item['history']:
        history_entry = {
            'nmID': nmID,
            'imtName': imtName,
            'vendorCode': vendorCode,
            'dt': history['dt'],
            'openCardCount': history['openCardCount'],
            'addToCartCount': history['addToCartCount'],
            'addToCartConversion': history['addToCartConversion'],
            'ordersCount': history['ordersCount'],
            'ordersSumRub': history['ordersSumRub'],
            'cartToOrderConversion': history['cartToOrderConversion'],
            'buyoutsCount': history['buyoutsCount'],
            'buyoutsSumRub': history['buyoutsSumRub'],
            'buyoutPercent': history['buyoutPercent']
        }
        flattened_data.append(history_entry)

# Convert the flattened data to a DataFrame
df = pd.DataFrame(flattened_data)
print(df)

df_copy=df.copy()

Total unique nmId values: 245
Data retrieved successfully for batch 1
Data retrieved successfully for batch 2
Data retrieved successfully for batch 3
Data retrieved successfully for batch 4
Data retrieved successfully for batch 5
Data retrieved successfully for batch 6
Data retrieved successfully for batch 7
Data retrieved successfully for batch 8
Data retrieved successfully for batch 9
Data retrieved successfully for batch 10
Data retrieved successfully for batch 11
Data retrieved successfully for batch 12
Data retrieved successfully for batch 13
           nmID                                            imtName vendorCode  \
0     133358869                   Чайник электрический KLF03 1,7 л  KLF03CREU   
1     133358869                   Чайник электрический KLF03 1,7 л  KLF03CREU   
2     133358869                   Чайник электрический KLF03 1,7 л  KLF03CREU   
3     133358869                   Чайник электрический KLF03 1,7 л  KLF03CREU   
4     133358869                   Чайник 

In [81]:
df_copy=df.copy()

In [82]:
df_copy

,nmID,imtName,vendorCode,dt,openCardCount,addToCartCount,addToCartConversion,ordersCount,ordersSumRub,cartToOrderConversion,buyoutsCount,buyoutsSumRub,buyoutPercent
0,133358869,"Чайник электрический KLF03 1,7 л",KLF03CREU,2025-03-06,3,0,0,0,0,0,0,0,0
1,133358869,"Чайник электрический KLF03 1,7 л",KLF03CREU,2025-03-07,0,0,0,0,0,0,0,0,0
2,133358869,"Чайник электрический KLF03 1,7 л",KLF03CREU,2025-03-08,3,0,0,0,0,0,0,0,0
3,133358869,"Чайник электрический KLF03 1,7 л",KLF03CREU,2025-03-09,3,1,33,0,0,0,0,0,0
4,133358869,"Чайник электрический KLF03 1,7 л",KLF03CREU,2025-03-10,5,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,159488077,Массажер для тела электрический Nozomi MH-103 ...,I01476,2025-03-07,318,13,4,0,0,0,0,0,0
1466,159488077,Массажер для тела электрический Nozomi MH-103 ...,I01476,2025-03-08,354,11,3,1,18000,9,0,0,0
1467,159488077,Массажер для тела электрический Nozomi MH-103 ...,I01476,2025-03-09,693,24,3,1,18000,4,0,0,0
1468,159488077,Массажер для тела электрический Nozomi MH-103 ...,I01476,2025-03-10,494,21,4,1,18000,5,0,0,0


In [83]:
df_unique = df_copy['ordersSumRub'].unique()
print(df_unique)

[     0   8190  15534   3425   3645   7485   3242   8105   4812   9624
   3208   7340   2277   4120   4110   2520   5040   2521   4936   2468
   4882   1794   1775   1487   2944   1472   2040  10186  20372  21523
  43046   2485   4970   7455   3447   1055   2110   3165   4170   8484
   2121   4242  18680   7472   3736  14944  29888  14856   2476  29713
  29712  17332   1363   2726   2470  15920   3576   2384   1192   7152
  20000   1392   2784   1742   1724   3452   6896  15279  30558   6130
   6060  21920   4669   4635   6180   3090   1185   2370  73720  81480
  62080 104760   7776  14978  22467   7489   2709  10836   8127   6320
  13080   6540   4265   8704  17408   4352   7900   8067   2689  13445
  16456   4114   8228  24684  12342   2751   5502  11004  30261   1625
  12686   6618   6610  44120  15442   8824   9981  46896  90861  96723
 126033  93792  22656  56663  15104  30208  33984  75520    534   1285
  58260   2573   2085  16168   8084   4026  12900  39895  31916   7979
  1595

In [84]:
df_copy

,nmID,imtName,vendorCode,dt,openCardCount,addToCartCount,addToCartConversion,ordersCount,ordersSumRub,cartToOrderConversion,buyoutsCount,buyoutsSumRub,buyoutPercent
0,133358869,"Чайник электрический KLF03 1,7 л",KLF03CREU,2025-03-06,3,0,0,0,0,0,0,0,0
1,133358869,"Чайник электрический KLF03 1,7 л",KLF03CREU,2025-03-07,0,0,0,0,0,0,0,0,0
2,133358869,"Чайник электрический KLF03 1,7 л",KLF03CREU,2025-03-08,3,0,0,0,0,0,0,0,0
3,133358869,"Чайник электрический KLF03 1,7 л",KLF03CREU,2025-03-09,3,1,33,0,0,0,0,0,0
4,133358869,"Чайник электрический KLF03 1,7 л",KLF03CREU,2025-03-10,5,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,159488077,Массажер для тела электрический Nozomi MH-103 ...,I01476,2025-03-07,318,13,4,0,0,0,0,0,0
1466,159488077,Массажер для тела электрический Nozomi MH-103 ...,I01476,2025-03-08,354,11,3,1,18000,9,0,0,0
1467,159488077,Массажер для тела электрический Nozomi MH-103 ...,I01476,2025-03-09,693,24,3,1,18000,4,0,0,0
1468,159488077,Массажер для тела электрический Nozomi MH-103 ...,I01476,2025-03-10,494,21,4,1,18000,5,0,0,0


In [79]:
# Convert 'day' and 'dt' to datetime for accurate merging
df_final_copy['day'] = pd.to_datetime(df_final_copy['day'])
df_copy['dt'] = pd.to_datetime(df['dt'])
# Rename columns in df2 to match df1 for merging
df_copy.rename(columns={'nmID': 'nmId', 'dt': 'day'}, inplace=True)

# Merge the DataFrames on 'nmId', 'day', and 'Project'
merged_df = pd.merge(
    df_final_copy,
    df_copy[['nmId', 'day', 'ordersCount', 'ordersSumRub']],
    on=['nmId', 'day'],
    how='left'
)

# Fill NaN values with 0 for ordersCount and ordersSumRub
merged_df['ordersCount'].fillna(0, inplace=True)
merged_df['ordersSumRub'].fillna(0, inplace=True)

# Display the merged DataFrame
merged_df

C:\Users\User\AppData\Local\Temp\ipykernel_14284\1478111281.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['ordersCount'].fillna(0, inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_14284\1478111281.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

,nmId,day,name_product,views,clicks,sum,atbs,orders,shks,sum_price,...,Project,Marketplace,endTime,createTime,startTime,name_campaign,status,type,ordersCount,ordersSumRub
0,25623718,2025-03-01 03:00:00,Бритва мужская электрическая S3 300BT с триммером,411,18,78.32,2,0,0,0,...,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания,0.0,0.0
1,25624340,2025-03-01 03:00:00,Электробритва Series 3 300s,444,20,79.94,1,0,0,0,...,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания,0.0,0.0
2,25628178,2025-03-01 03:00:00,Бритва мужская электрическая S3 300s,474,17,83.18,0,0,0,0,...,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания,0.0,0.0
3,25725047,2025-03-01 03:00:00,Триммер для бороды и усов BT3222,2,0,0.10,0,0,0,0,...,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Триммеры_07.11.2024,Идут показы,Автоматическая кампания,0.0,0.0
4,25962484,2025-03-01 03:00:00,Кухонные весы Kitchen Energy,1104,17,170.52,0,0,0,0,...,WB-GutenTech,Wildberries,2100-01-01,2024-07-25,2025-01-09,CASO_Весы кух_2 SKU_25.07.24,Кампания на паузе,Автоматическая кампания,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4228,288465481,2025-03-10 03:00:00,Набор Gillette Venus с чехлом для бритвы,1427,93,208.60,2,1,1,1115,...,WB-Smart-Market,Wildberries,2100-01-01,2025-02-12,2025-03-05,VENUS_Наборы_12.02.2025,Кампания на паузе,Автоматическая кампания,0.0,0.0
4229,288465487,2025-03-10 03:00:00,Набор Series Пена для бритья 200мл + Мочалка,1,0,0.06,0,0,0,0,...,WB-Smart-Market,Wildberries,2100-01-01,2025-02-12,2025-02-24,Gillette_Бритвы + наборы_12.02.2025,Кампания на паузе,Автоматическая кампания,0.0,0.0
4230,313543155,2025-03-10 03:00:00,,1272,18,164.92,3,2,2,6906,...,WB-Smart-Market,Wildberries,2100-01-01,2025-02-12,2025-02-21,Gillette_Кассеты_12.02.2025,Кампания на паузе,Автоматическая кампания,0.0,0.0
4231,313543156,2025-03-10 03:00:00,,1542,22,203.51,4,1,1,3664,...,WB-Smart-Market,Wildberries,2100-01-01,2025-02-12,2025-02-21,Gillette_Кассеты_12.02.2025,Кампания на паузе,Автоматическая кампания,0.0,0.0


In [85]:
df_unique = merged_df['ordersSumRub'].unique()
print(df_unique)

[0.]


## Insert the data

In [ ]:
password = os.getenv('ClickHouse')
# Define connection parameters
client = get_client(
    host='rc1a-vk5i3icccvmfk6cm.mdb.yandexcloud.net',  # Your Yandex Cloud ClickHouse host
    port=8443,                                          # Yandex Cloud uses port 8443 for HTTPS
    username='user1',                                      # Your ClickHouse username
    password= password,                           # Your ClickHouse password
    database='user1',                            # Your database name
    secure=True,                                        # Use HTTPS
    verify=False                                        # Disable SSL certificate verification 
    # Define the data to insert
)

In [ ]:
""" # Define the table name
table_name = 'campaign_data_wb'

# Convert DataFrame to a list of tuples for bulk insertion
data = [tuple(row) for row in df_final.to_numpy()]

# Define the column names
column_names = [
    'nmId', 'day', 'name_product', 'views', 'clicks', 'sum', 'atbs', 'orders', 'shks', 'sum_price',
    'advertId', 'Project', 'Marketplace', 'endTime', 'createTime', "startTime", 'name_campaign', 'status', 'type'
]

# Use the insert method for bulk insertion
client.insert(table_name, data, column_names=column_names)

print("Data inserted successfully!") """

Data inserted successfully!


In [ ]:
merget_df_copy=merged_df.copy()
merget_df_copy


' merget_df_copy=merged_df.copy()\nmerget_df_copy\n '

In [ ]:

# Ensure date columns are in the correct format for ClickHouse
merget_df_copy['day'] = pd.to_datetime(merget_df_copy['day'])  # Convert to datetime


# Debugging: Check the data types of the DataFrame
print("Data types of merged_df:")
print(merget_df_copy.dtypes)

# Ensure the DataFrame has the correct columns
columns = [
    'nmId', 'day', 'name_product', 'views', 'clicks', 'sum', 'atbs', 'orders', 'shks',
    'ordersCount', 'ordersSumRub', 'sum_price', 'advertId', 'Project', 'Marketplace',
    'endTime', 'createTime', 'startTime', 'name_campaign', 'status', 'type'
]

# Reorder columns to match the expected order
merget_df_copy_2 = merget_df_copy[columns]

# Convert DataFrame to a list of tuples for bulk insertion
data = [tuple(row) for row in merget_df_copy_2.to_numpy()]

# Debugging: Check the structure of the data
print("Sample data to insert:", data[:5])  # Print the first 5 rows to check the structure

# Define the table name
table_name = 'campaign_data_wb_new'

# Use the insert method for bulk insertion
client.insert(table_name, data, column_names=columns)
print("Data inserted successfully!")

' \n# Ensure date columns are in the correct format for ClickHouse\nmerget_df_copy[\'day\'] = pd.to_datetime(merget_df_copy[\'day\'])  # Convert to datetime\n\n\n# Debugging: Check the data types of the DataFrame\nprint("Data types of merged_df:")\nprint(merget_df_copy.dtypes)\n\n# Ensure the DataFrame has the correct columns\ncolumns = [\n    \'nmId\', \'day\', \'name_product\', \'views\', \'clicks\', \'sum\', \'atbs\', \'orders\', \'shks\',\n    \'ordersCount\', \'ordersSumRub\', \'sum_price\', \'advertId\', \'Project\', \'Marketplace\',\n    \'endTime\', \'createTime\', \'startTime\', \'name_campaign\', \'status\', \'type\'\n]\n\n# Reorder columns to match the expected order\nmerget_df_copy_2 = merget_df_copy[columns]\n\n# Convert DataFrame to a list of tuples for bulk insertion\ndata = [tuple(row) for row in merget_df_copy_2.to_numpy()]\n\n# Debugging: Check the structure of the data\nprint("Sample data to insert:", data[:5])  # Print the first 5 rows to check the structure\n\n# De